# Looking at specific paths:

Let's look more closely at forced paths. Can we even project onto the exact points we want to?

In [ ]:
import batch_test_latent_sim as bt
import numpy as np
from matplotlib import pylab as plt

In [ ]:
def eval_path_via_inversion(ls, T1,p1, T2,p2):
    states=[]
    ss = np.linspace(0.0,1.0,50)
    for s in ss:
        try:
#             pp=p1*(1.0-(np.exp(s)-1)/(np.e-1) )  + p2*((np.exp(s)-1)/(np.e-1))
            pp = np.exp( np.log(p1)*(1.0-s) + np.log(p2)*s )
            print(pp)
            q0 = ls.find_point(T=T1*(1.0-s)+T2*s, 
                               p=pp,
                              under_relax=0.1)
            state = ls.decode(q0)
            states.append(state)
        except Exception as e:
            print(e)
    return np.vstack(states)
def eval_path_via_latent_space(ls, T1,p1, T2,p2):
    states=[]
    ss = np.linspace(0.0,1.0,50)
    q1 = ls.find_point(T=T1, p=p1)
    q2 = ls.find_point(T=T2, p=p2)
    for s in ss:
        try:
            state = ls.decode(q1*(1.0-s)+q2*s)
            states.append(state)
        except Exception as e:
            print(e)
            
    return np.vstack(states)
def plot_contour(ls, problem,method='inversion',xidx=1,yidx=2):
    if method=='inversion':
        states = eval_path_via_inversion(ls, problem.initial['T'],problem.initial['p'],
                          problem.answer['T'],problem.answer['p'])
    else:
        states = eval_path_via_latent_space(ls, problem.initial['T'],problem.initial['p'],
                          problem.answer['T'],problem.answer['p'])
    plt.semilogx(states[:,xidx],states[:,yidx],'+-',label=ls.model_name)
    
    return states


In [ ]:
states = []
nets = [
    'Classifying_1,3,6,12,tanh',
    'Classifying_1,3,6,12,sigmoid',
    'Classifying_1,5,6,12,sigmoid'
]
for n in nets:
    ls = bt.curried_latentssim('water_lg',n)
    states.append(plot_contour(ls,bt.Transition_L2G,method='inversion'))
    
nets = [
    'Classifying_2,3,12,24,sigmoid',
    'Classifying_2,4,12,24,sigmoid (1)',
#     'Classifying_1,5,6,12,sigmoid'
]

for n in nets:
    ls = bt.curried_latentssim('water_slgc_logp_64',n)
    states.append(plot_contour(ls,bt.Transition_L2G,method='inversion'))
    
plt.legend()

In [ ]:
states[0]

In [ ]:
states[1]